# FAIR workbench demo using `mark_as_fairstep` decorator and code injection 
In this notebook we demo using the `mark_as_fairstep` decorator to mark a function as a FAIR step, this automatically extracts information from the function definition as well as storing the raw code in the `description` of the step. We can then inject this code in the python notebook and execute it using the notebook.

## Advantages:
* Simple and easy-to-use (at least for the happy flow)
* By letting the user execute the step, we leave it up to the user to think about security and actually making the code work.

## Disadvantages:
* Highly irreproducible, since we only store the code and not all dependencies (i.e. python libraries, environment)
* Very buggy, parsing the function definition for filling the RDF is a tricky thing. It will work for the happy flow but it is hard to get it right for all edgecases. Some foreseen problems:
 - Indentation
 - Non-pure functions
 - Unexpected input/return types
 - Dependence on other functions

In [1]:
import fairworkflows
print(fairworkflows.__version__)

0.2.0


In [2]:
from fairworkflows import FairStep, FairWorkflow, mark_as_fairstep

## Use the `mark_as_fairstep` decorator to mark a function as a FAIR step

## The `mark_as_fairstep` decorator triggers validation
In the following case we are missing a label and we forgot to say whether it is a script or manual task, so an error will be generated:

In [3]:
@mark_as_fairstep()
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

AssertionError: Step RDF has no rdfs:label
Step RDF must be either a bpmn:ManualTask or a bpmn:ScriptTask


## Mark functions as fair steps
In the following, we provide a label and script task type, so that the generated FairStep validates:

In [4]:
@mark_as_fairstep(label='add integers', is_script_task=True)
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

In [5]:
@mark_as_fairstep(label='square root', is_script_task=True)
def sqrt(a: int) -> float:
    """
    Computational step taking the square root of an integer.
    """
    return a ** 0.5

## Publish using the API
Let's publish step 1 to see the resulting RDF. One could use the notebook extension, it doesn't have the newest features yet though.
Be sure to checkout the resulting RDF, you'll see we extract a lot of triples from the function definition. http://purl.org/np/RAcQHJ_Nxq6cUQkzg8J-xOiBeRee5aLQXf_fAebFwQDks

In [6]:
step1 = FairStep.from_function(add)
step1.publish_as_nanopub(use_test_server=True)

Published to http://purl.org/np/RA5DxCRDIYX_IWqcHRz1-No-xq-svp1KjLiX-MKDZadyY
Published concept to http://purl.org/np/RA5DxCRDIYX_IWqcHRz1-No-xq-svp1KjLiX-MKDZadyY#step


{'nanopub_uri': 'http://purl.org/np/RA5DxCRDIYX_IWqcHRz1-No-xq-svp1KjLiX-MKDZadyY',
 'concept_uri': 'http://purl.org/np/RA5DxCRDIYX_IWqcHRz1-No-xq-svp1KjLiX-MKDZadyY#step'}

## Define the workflow

In [7]:
workflow = FairWorkflow(description='A simple workflow', label='test workflow')
step1 = FairStep.from_function(add)
step2 = FairStep.from_function(sqrt)
workflow.add(step1)
workflow.add(step2, follows=step1)

## Execution of steps
Execution of steps is just done by calling the functions

In [8]:
intermediate_result = add(1, 3)
sqrt(intermediate_result)

2.0

You can inject raw code from steps fetched from nanopub:

In [9]:
@mark_as_fairstep(label='add integers', is_script_task=True)
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b

And just call the function:

In [10]:
result = add(1, 3)